## Agents in Team

A single Agent can only say create a short story for us.
but with a team whre many agents work together towards a common goal they can help us in writing or even helping to review, edit etc.

In [1]:
import asyncio
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
model_client = OpenAIChatCompletionClient(model='gpt-4o', api_key=api_key)

In [2]:
model_client_2 = OpenAIChatCompletionClient(model='gpt-4',api_key=api_key)

In [3]:
from autogen_agentchat.agents import AssistantAgent

dsa_solver = AssistantAgent(
    name = 'Complex_DSA_Solver',
    model_client=model_client,
    description='A DSA solver',
    system_message="You give code in python to solve complex DSA problems. Give under 100 words"
)

code_reviewer = AssistantAgent(
    name = 'CODE_REVEIWER',
    model_client=model_client_2,
    description='A Code Reviewer',
    system_message="You review the code given by the complex_dsa_solver and make sure it is optimized.Give under 10 words"
)

code_editor = AssistantAgent(
    name = 'CODE_EDITOR',
    model_client=model_client,
    description='A Code editor',
    system_message="You make the code easy to understand and add comments wherever required.Give under 10 words"
)

In [6]:
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.messages import TextMessage

team = RoundRobinGroupChat(
    participants=[dsa_solver,code_reviewer,code_editor],
    max_turns=15 # -----># maximum number of Message before it stops between the agents.
)

async def test_team():
    task = TextMessage(content='Write a simple Hello world code ?',source='User')
    
    result = await team.run(task=task)

    for each_agent_message in result.messages:
        print(f'{((each_agent_message))} ' )
        print('\n \n')


await test_team()

id='7245ac95-f088-46e1-ac93-43c40e81794f' source='User' models_usage=None metadata={} created_at=datetime.datetime(2025, 7, 12, 7, 29, 8, 473882, tzinfo=datetime.timezone.utc) content='Write a simple Hello world code ?' type='TextMessage' 

 

id='b65c762d-e185-486a-a5f4-07d2446b4d62' source='Complex_DSA_Solver' models_usage=RequestUsage(prompt_tokens=438, completion_tokens=10) metadata={} created_at=datetime.datetime(2025, 7, 12, 7, 29, 9, 46587, tzinfo=datetime.timezone.utc) content='```python\nprint("Hello, world!")\n```' type='TextMessage' 

 

id='50d6194f-03a8-4ccd-8ce0-1e8bcfe5e7ef' source='CODE_REVEIWER' models_usage=RequestUsage(prompt_tokens=453, completion_tokens=10) metadata={} created_at=datetime.datetime(2025, 7, 12, 7, 29, 10, 664476, tzinfo=datetime.timezone.utc) content="Optimization isn't required, code is efficient." type='TextMessage' 

 

id='63010243-ca33-4b56-85a6-3b9a221618f0' source='CODE_EDITOR' models_usage=RequestUsage(prompt_tokens=511, completion_tokens=9)

In [7]:
await team.reset()  # Reset the team for a new task.
from autogen_agentchat.base import TaskResult

async for message in team.run_stream(task="Write a simple Hello world code ?"):  # type: ignore
    if isinstance(message, TaskResult):
        print("Stop Reason:", message.stop_reason)
    else:
        print(message.source,message)

user id='cb7692c5-a289-42b3-9f64-cba006ad8f68' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 7, 12, 7, 30, 8, 972112, tzinfo=datetime.timezone.utc) content='Write a simple Hello world code ?' type='TextMessage'
Complex_DSA_Solver id='dc4d346b-f5ae-4e9a-a560-bd9891b77627' source='Complex_DSA_Solver' models_usage=RequestUsage(prompt_tokens=36, completion_tokens=42) metadata={} created_at=datetime.datetime(2025, 7, 12, 7, 30, 9, 918632, tzinfo=datetime.timezone.utc) content='Here is a simple "Hello, World!" program in Python:\n\n```python\nprint("Hello, World!")\n```\n\nSimply run this code, and it will display "Hello, World!" in the console.' type='TextMessage'
CODE_REVEIWER id='65d96dc4-3d1e-4460-88c9-3c28167ba13c' source='CODE_REVEIWER' models_usage=RequestUsage(prompt_tokens=94, completion_tokens=7) metadata={} created_at=datetime.datetime(2025, 7, 12, 7, 30, 11, 457773, tzinfo=datetime.timezone.utc) content='Code is concise and efficient enough.' type

In [8]:
from autogen_agentchat.ui import Console
await team.reset()  # Reset the team for a new task.
await Console(team.run_stream(task="Write a simple Hello world code."))  # Stream the messages to the console.

---------- TextMessage (user) ----------
Write a simple Hello world code.
---------- TextMessage (Complex_DSA_Solver) ----------
Sure! Here is a simple "Hello, World!" program in Python:

```python
print("Hello, World!")
```

This code uses the `print()` function to display the message "Hello, World!" on the screen.
---------- TextMessage (CODE_REVEIWER) ----------
Code is already optimized.
---------- TextMessage (CODE_EDITOR) ----------
Great to hear! It's simple and clear.
---------- TextMessage (Complex_DSA_Solver) ----------
If you have any other code or need further help, feel free to ask!
---------- TextMessage (CODE_REVEIWER) ----------
Understood. Waiting for further instructions.
---------- TextMessage (CODE_EDITOR) ----------
Sure! Ready to assist whenever needed.
---------- TextMessage (Complex_DSA_Solver) ----------
Just let me know if you need any more help or code editing!
---------- TextMessage (CODE_REVEIWER) ----------
Code is already at maximum efficiency.
----------

TaskResult(messages=[TextMessage(id='94b6d234-3a6c-4766-8cff-171bc9956084', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 12, 7, 30, 55, 463122, tzinfo=datetime.timezone.utc), content='Write a simple Hello world code.', type='TextMessage'), TextMessage(id='f0b179fc-74f9-4fc8-b706-cfeab25cdf48', source='Complex_DSA_Solver', models_usage=RequestUsage(prompt_tokens=36, completion_tokens=47), metadata={}, created_at=datetime.datetime(2025, 7, 12, 7, 31, 0, 218272, tzinfo=datetime.timezone.utc), content='Sure! Here is a simple "Hello, World!" program in Python:\n\n```python\nprint("Hello, World!")\n```\n\nThis code uses the `print()` function to display the message "Hello, World!" on the screen.', type='TextMessage'), TextMessage(id='a1c8ed89-8ff5-4ab2-8436-1be800827bda', source='CODE_REVEIWER', models_usage=RequestUsage(prompt_tokens=99, completion_tokens=5), metadata={}, created_at=datetime.datetime(2025, 7, 12, 7, 31, 1, 178182, tzinfo=datetime.time

In [9]:
await Console(team.run_stream(task="conitnue."))

---------- TextMessage (user) ----------
conitnue.
---------- TextMessage (Complex_DSA_Solver) ----------
Please provide more information or let me know what you would like to continue with. I'm here to help!
---------- TextMessage (CODE_REVEIWER) ----------
Awaiting for further instructions.
---------- TextMessage (CODE_EDITOR) ----------
Ready to assist whenever you're ready!
---------- TextMessage (Complex_DSA_Solver) ----------
Just let me know how I can help further!
---------- TextMessage (CODE_REVEIWER) ----------
No optimization needed. Code is perfect.
---------- TextMessage (CODE_EDITOR) ----------
Great! Let me know if you need anything.
---------- TextMessage (Complex_DSA_Solver) ----------
Feel free to reach out if you need more help!
---------- TextMessage (CODE_REVEIWER) ----------
Already optimized. Ready for next task.
---------- TextMessage (CODE_EDITOR) ----------
All set! Ready for your next request.
---------- TextMessage (Complex_DSA_Solver) ----------
Great! Just

TaskResult(messages=[TextMessage(id='83a73b11-b0b2-4d9d-bd0c-01185c500211', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 12, 7, 31, 43, 402008, tzinfo=datetime.timezone.utc), content='conitnue.', type='TextMessage'), TextMessage(id='e22fa871-2a24-49b3-a4b8-79bc84ab4f03', source='Complex_DSA_Solver', models_usage=RequestUsage(prompt_tokens=332, completion_tokens=21), metadata={}, created_at=datetime.datetime(2025, 7, 12, 7, 31, 44, 294570, tzinfo=datetime.timezone.utc), content="Please provide more information or let me know what you would like to continue with. I'm here to help!", type='TextMessage'), TextMessage(id='cc7f62f7-e4cd-4a3b-888e-3d41336cef2f', source='CODE_REVEIWER', models_usage=RequestUsage(prompt_tokens=372, completion_tokens=6), metadata={}, created_at=datetime.datetime(2025, 7, 12, 7, 31, 45, 657467, tzinfo=datetime.timezone.utc), content='Awaiting for further instructions.', type='TextMessage'), TextMessage(id='970a4105-d23a-4c6

In [ ]:
1. User Proxy 

2. Termination Condition

In [ ]:
print()

In [10]:
from autogen_agentchat.agents import AssistantAgent
add_1_agent_first = AssistantAgent(
    name = 'add_1_agent_first',
    model_client=model_client,
    system_message="Add 1 to the number, first number is 0. Give result as output"
)

add_1_agent_second = AssistantAgent(
    name = 'add_1_agent_second',
    model_client=model_client,
    system_message="Add 1 to the number you got from previous run. Give result as output."
)
 
add_1_agent_third = AssistantAgent(
    name = 'add_1_agent_third',
    model_client=model_client,
    system_message="Add 1 to the number from previous run. Give result as output."
)

In [16]:
my_increment_team = RoundRobinGroupChat(participants=[add_1_agent_first,add_1_agent_second,add_1_agent_third],max_turns=2)


### Resume a Team

In [20]:
await Console(my_increment_team.run_stream())

---------- TextMessage (add_1_agent_first) ----------
16
---------- TextMessage (add_1_agent_second) ----------
17


TaskResult(messages=[TextMessage(id='8648b0c9-a7a8-4359-a5e5-2fee82bb6592', source='add_1_agent_first', models_usage=RequestUsage(prompt_tokens=159, completion_tokens=1), metadata={}, created_at=datetime.datetime(2025, 7, 12, 7, 34, 44, 14539, tzinfo=datetime.timezone.utc), content='16', type='TextMessage'), TextMessage(id='d97a132e-42d0-45cf-bd0c-cbb9287c59b9', source='add_1_agent_second', models_usage=RequestUsage(prompt_tokens=179, completion_tokens=1), metadata={}, created_at=datetime.datetime(2025, 7, 12, 7, 34, 44, 511486, tzinfo=datetime.timezone.utc), content='17', type='TextMessage')], stop_reason='Maximum number of turns 2 reached.')

In [18]:
await Console(my_increment_team.run_stream())

---------- TextMessage (add_1_agent_third) ----------
12
---------- TextMessage (add_1_agent_first) ----------
13


TaskResult(messages=[TextMessage(id='23c0929e-9ea1-4f35-82a6-056957b25482', source='add_1_agent_third', models_usage=RequestUsage(prompt_tokens=142, completion_tokens=1), metadata={}, created_at=datetime.datetime(2025, 7, 12, 7, 34, 7, 369033, tzinfo=datetime.timezone.utc), content='12', type='TextMessage'), TextMessage(id='5dcd766a-4602-4e53-99cd-05a7f6fd4282', source='add_1_agent_first', models_usage=RequestUsage(prompt_tokens=133, completion_tokens=1), metadata={}, created_at=datetime.datetime(2025, 7, 12, 7, 34, 7, 939123, tzinfo=datetime.timezone.utc), content='13', type='TextMessage')], stop_reason='Maximum number of turns 2 reached.')

In [19]:
await team.reset()
await Console(my_increment_team.run_stream())

---------- TextMessage (add_1_agent_second) ----------
14
---------- TextMessage (add_1_agent_third) ----------
15


TaskResult(messages=[TextMessage(id='77ae90be-31e7-40d0-8a42-c600530436b5', source='add_1_agent_second', models_usage=RequestUsage(prompt_tokens=153, completion_tokens=1), metadata={}, created_at=datetime.datetime(2025, 7, 12, 7, 34, 10, 635026, tzinfo=datetime.timezone.utc), content='14', type='TextMessage'), TextMessage(id='fb7fd9b9-2c06-4d9d-92e1-609e32b98c5c', source='add_1_agent_third', models_usage=RequestUsage(prompt_tokens=167, completion_tokens=1), metadata={}, created_at=datetime.datetime(2025, 7, 12, 7, 34, 11, 301615, tzinfo=datetime.timezone.utc), content='15', type='TextMessage')], stop_reason='Maximum number of turns 2 reached.')

In [21]:
await my_increment_team.reset()
await Console(my_increment_team.run_stream())

---------- TextMessage (add_1_agent_first) ----------
1
---------- TextMessage (add_1_agent_second) ----------
2


TaskResult(messages=[TextMessage(id='c4b21a1c-0c5e-42e2-838d-4ae041efb0a0', source='add_1_agent_first', models_usage=RequestUsage(prompt_tokens=24, completion_tokens=1), metadata={}, created_at=datetime.datetime(2025, 7, 12, 7, 35, 19, 687882, tzinfo=datetime.timezone.utc), content='1', type='TextMessage'), TextMessage(id='eb6e13f1-094c-4720-8367-fb48ac2fdf69', source='add_1_agent_second', models_usage=RequestUsage(prompt_tokens=34, completion_tokens=1), metadata={}, created_at=datetime.datetime(2025, 7, 12, 7, 35, 20, 225211, tzinfo=datetime.timezone.utc), content='2', type='TextMessage')], stop_reason='Maximum number of turns 2 reached.')